---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [328]:
import re
import pandas as pd
import numpy as np

def date_sorter():
    
    doc = []
    with open('dates.txt') as file:
        for line in file:
            doc.append(line)

    df = pd.Series(doc)

    df = df.to_frame()
    df['orig_index'] = df.index
    df.columns = ['text','orig_index']
    #df.head(10)
    
    # Mapping for month to integer
    d = {'JAN':1, 'FEB':2, 'MAR':3, 'APR':4, 'MAY':5, 'JUN':6, 'JUL':7, 'AUG':8, 'SEP':9, 'OCT':10, 'NOV':11, 'DEC':12,
        'JANUARY':1,'FEBRUARY':2, 'MARCH':3, 'APRIL':4, 'JUNE':6, 'JULY':7, 'AUGUST':8, 'SEPTEMBER':9, 'OCTOBER':10, 
         'NOVEMBER':11, 'DECEMBER':12, 'DECEMEBER':12, 'JANAURY':1}
    
    #### extract the strings in format month day year with month in digits
    dates_1 = df.text.str.extractall(r'((?P<month>\d{1,2})[-\/](?P<day>\d{1,2})[/\-](?P<year>\d{2,4}))')
    dates_1['month'] = dates_1['month'].apply('{:0>2}'.format) 
    dates_1['day'] = dates_1['day'].apply('{:0>2}'.format)
    dates_1['year'] = np.where(dates_1['year'].str.len()==2, '19' + dates_1['year'],dates_1['year'])
    dates_1.reset_index(inplace=True)
    dates_1['orig_index'] = dates_1['level_0']
    
    # get the remaining dates
    dates_1['identified'] = 1
    dates_merge = dates_1[['orig_index','identified']]
    remaining_text = pd.merge(df,dates_merge,how='outer', on = 'orig_index')
    remaining_text = remaining_text[remaining_text['identified'] != 1 ]
    remaining_text.index = remaining_text['orig_index']
    del remaining_text['identified']
    
    
    #### extract the strings in format day month year with month in words
    dates_2 = remaining_text.text.str.extractall(r'((\d{1,2}) ((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*) (\d{4}))')
    dates_2.columns = ['0','day','month','year']
    dates_2['month'] = dates_2['month'].str.upper()
    dates_2['month'] = dates_2['month'].map(d)
    dates_2['month'] = dates_2['month'].astype(str)
    dates_2['month'] = np.where(dates_2['month'].str.len()==1, '0' + dates_2['month'],dates_2['month'])
    dates_2['day'] = dates_2['day'].fillna('01')
    dates_2.reset_index(inplace=True)
    
    # get the remaining dates
    dates_2['identified'] = 1
    dates_merge = dates_2[['orig_index','identified']]
    remaining_text = pd.merge(remaining_text,dates_merge,how='outer', on = 'orig_index')
    remaining_text = remaining_text[remaining_text['identified'] != 1 ]
    remaining_text.index = remaining_text['orig_index']
    del remaining_text['identified']
    
    #### Get the rows that take the form month then day then four digit year
    dates_3 = remaining_text.text.str.extractall(r'(((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[?:.,]* (\d{1,2})[?:.,]* (\d{4}))')
    dates_3.columns = ['0','month','day','year']
    dates_3['month'] = dates_3['month'].str.upper()
    dates_3['month'] = dates_3['month'].map(d)
    dates_3['month'] = dates_3['month'].astype(str)
    dates_3['month'] = np.where(dates_3['month'].str.len()==1, '0' + dates_3['month'],dates_3['month'])
    dates_3.reset_index(inplace=True)
    
    dates_3['identified'] = 1
    dates_merge = dates_3[['orig_index','identified']]
    remaining_text = pd.merge(remaining_text,dates_merge,how='outer', on = 'orig_index')
    remaining_text = remaining_text[remaining_text['identified'] != 1 ]
    remaining_text.index = remaining_text['orig_index']
    del remaining_text['identified']
    
    #### Get the rows that have only a month and a year
    dates_4 = remaining_text.text.str.extractall(r'(((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[?:.,]* (\d{4}))')
    dates_4.columns = ['0','month','year']
    dates_4['month'] = dates_4['month'].str.upper()
    dates_4['month'] = dates_4['month'].map(d)
    dates_4['month'] = dates_4['month'].astype(str)
    dates_4['month'] = np.where(dates_4['month'].str.len()==1, '0' + dates_4['month'],dates_4['month'])
    dates_4['day'] = '01'
    dates_4.reset_index(inplace=True)
    
    dates_4['identified'] = 1
    dates_merge = dates_4[['orig_index','identified']]
    remaining_text = pd.merge(remaining_text,dates_merge,how='outer', on = 'orig_index')
    remaining_text = remaining_text[remaining_text['identified'] != 1 ]
    remaining_text.index = remaining_text['orig_index']
    del remaining_text['identified']
    
    #### Get the rows that have only a month and year in the format mm/yyyy
    dates_5 = remaining_text.text.str.extractall(r'((\d{1,2})[/](\d{4}))')
    dates_5.columns = ['0','month','year']
    dates_5['month'] = dates_5['month'].astype(str)
    dates_5['month'] = np.where(dates_5['month'].str.len()==1, '0' + dates_5['month'],dates_5['month'])
    dates_5['day'] = '01'
    dates_5.reset_index(inplace=True)
    dates_5['identified'] = 1
    
    dates_merge = dates_5[['orig_index','identified']]
    remaining_text = pd.merge(remaining_text,dates_merge,how='outer', on = 'orig_index')
    remaining_text = remaining_text[remaining_text['identified'] != 1 ]
    remaining_text.index = remaining_text['orig_index']
    del remaining_text['identified']
    
    #### Everything that is left should be year only
    dates_6 = remaining_text.text.str.extractall(r'((\d{4}))')
    dates_6.columns = ['0','year']
    dates_6['day'] = '01'
    dates_6['month'] = '01'
    dates_6.reset_index(inplace=True)
    dates_6['identified'] = 1
    
    dates_merge = dates_6[['orig_index','identified']]
    remaining_text = pd.merge(remaining_text,dates_merge,how='outer', on = 'orig_index')
    remaining_text = remaining_text[remaining_text['identified'] != 1 ]
    remaining_text.index = remaining_text['orig_index']
    del remaining_text['identified']
    
    frames = [dates_1,dates_2,dates_3,dates_4,dates_5,dates_6]
    parsed_dates = pd.concat(frames)
    parsed_dates = parsed_dates.sort_values(by=['orig_index'])
    parsed_dates.reset_index(inplace=True)
    
    parsed_dates = parsed_dates.drop(parsed_dates.index[[73]])
    
    parsed_dates['final_date'] = parsed_dates['month'] + '/' + parsed_dates['day'] + '/' + parsed_dates['year']
    parsed_dates['final_date'] = pd.to_datetime(parsed_dates['final_date'])
    
    parsed_dates = parsed_dates.sort_values(by=['final_date'])
    parsed_dates = parsed_dates.drop('level_0',axis = 1)
    parsed_dates.reset_index(inplace=True)
    
    final_out = parsed_dates['orig_index']
    del final_out.index.name
    
    return pd.Series(final_out)

date_sorter()



0      9  
1      84 
2      2  
3      53 
4      28 
5      474
6      153
7      13 
8      129
9      98 
10     111
11     225
12     31 
13     171
14     191
15     486
16     335
17     415
18     36 
19     405
20     323
21     422
22     375
23     380
24     345
25     57 
26     481
27     436
28     104
29     299
30     162
31     154
32     402
33     95 
34     73 
35     108
36     156
37     332
38     182
39     82 
40     351
41     278
42     214
43     155
44     223
45     473
46     49 
47     317
48     11 
49     319
50     40 
51     418
52     165
53     370
54     382
55     3  
56     50 
57     363
58     219
59     465
60     237
61     23 
62     342
63     204
64     258
65     315
66     27 
67     93 
68     17 
69     488
70     303
71     283
72     395
73     309
74     419
75     123
76     19 
77     117
78     232
79     72 
80     189
81     369
82     493
83     318
84     239
85     148
86     105
87     336
88     6  
89     200
90     81 

Data Type Test (Series?): Passed
Data Shape Test ((500,)?): Passed
Index Values Test (range(500)?): Passed
Values Test (0-499): Passed

